In [2]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By

In [5]:
HEADERS = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control':'max-age=0',
    'Connection':'keep-alive',
    'Cookie':'browserid=3086693902866888623; timezoneOffset=28800,0; _ga=GA1.2.2138540995.1679213111; lastagecheckage=19-0-2000; Steam_Language=tchinese; steamLoginSecure=76561198861935902%7C%7CeyAidHlwIjogIkpXVCIsICJhbGciOiAiRWREU0EiIH0.eyAiaXNzIjogInI6MEQyRV8yMjNGM0MwNV9BRDBERCIsICJzdWIiOiAiNzY1NjExOTg4NjE5MzU5MDIiLCAiYXVkIjogWyAid2ViIiBdLCAiZXhwIjogMTY5MzEyNzAwMCwgIm5iZiI6IDE2ODQzOTkyMzEsICJpYXQiOiAxNjkzMDM5MjMxLCAianRpIjogIjBEMTRfMjMwRURDRjNfRjgzQTkiLCAib2F0IjogMTY3OTIzNjQ5MCwgInJ0X2V4cCI6IDE2OTc1Mjc1NDgsICJwZXIiOiAwLCAiaXBfc3ViamVjdCI6ICIzNi4yMzMuMTMxLjI0MCIsICJpcF9jb25maXJtZXIiOiAiMzYuMjMzLjEzMS4yNDAiIH0.gzSwhoeP-519zUXYSY5f9Imb-AOfdwh-CxyQD-ICA2xOJ9jQJ3QFXm3ssxr5Jz7_YbsgGQrCoDs6ad3MZXUKCA; sessionid=0a7de6c5b8475db44e05412a; birthtime=945532801; deep_dive_carousel_method=default; deep_dive_carousel_focused_app=524220; app_impressions=1293830@1_7_7_globaltopsellers_150_2|1648532@1_7_7_globaltopsellers_150_2|582660@1_7_7_globaltopsellers_150_2|431960@1_7_7_globaltopsellers_150_2|1364780@1_7_7_globaltopsellers_150_2|1222670@1_7_7_globaltopsellers_150_2|1142710@1_7_7_globaltopsellers_150_2|1384160@1_7_7_globaltopsellers_150_2|412020@1_7_7_globaltopsellers_150_2|1326470@1_7_7_globaltopsellers_150_2|671860@1_7_7_globaltopsellers_150_2|1272080@1_7_7_globaltopsellers_150_2|1868140@1_7_7_globaltopsellers_150_2|613100@1_7_7_globaltopsellers_150_2|990080@1_7_7_globaltopsellers_150_2|1973710@1_7_7_globaltopsellers_150_2|1260320@1_7_7_globaltopsellers_150_2|1971870@1_7_7_globaltopsellers_150_2|1151340@1_7_7_globaltopsellers_150_3|1811260@1_7_7_globaltopsellers_150_3|1049590@1_7_7_globaltopsellers_150_3|270880@1_7_7_globaltopsellers_150_3|1248130@1_7_7_globaltopsellers_150_3|1304930@1_7_7_globaltopsellers_150_3|108600@1_7_7_globaltopsellers_150_3|1174180@1_7_7_globaltopsellers_150_3|1172620@1_7_7_globaltopsellers_150_3|2325830@1_7_7_globaltopsellers_150_3|949230@1_7_7_globaltopsellers_150_3|552990@1_7_7_globaltopsellers_150_3|284160@1_7_7_globaltopsellers_150_3|311210@1_7_7_globaltopsellers_150_3|2357570@1_7_7_globaltopsellers_150_3|1945340@1_7_7_globaltopsellers_150_3|2451950@1_7_7_globaltopsellers_150_3|239140@1_7_7_globaltopsellers_150_3|1217060@1_7_7_globaltopsellers_150_3|527230@1_7_7_globaltopsellers_150_3|2195250@1_7_7_globaltopsellers_150_3; recentapps=%7B%222357570%22%3A1693106043%2C%221888160%22%3A1693105767%2C%221973710%22%3A1693105656%2C%221832960%22%3A1693104792%2C%221272080%22%3A1693104664%2C%222138330%22%3A1693104482%2C%222325830%22%3A1693104450%2C%221648532%22%3A1693104308%2C%222378500%22%3A1693104212%2C%221086940%22%3A1693104096%7D',
    'Host':'store.steampowered.com',
    'Sec-Fetch-Mode':'navigate',
    'Sec-Fetch-Site':'same-origin',
    'Sec-Fetch-User':'?1',
    'Upgrade-Insecure-Requests':'1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
        }

n = 100
path = 'steam_Link&ID.xlsx'
def getgamelist(n):
    linklist=[]
    IDlist = []
    for pagenum in range(1,n+1):
        r = requests.get('https://store.steampowered.com/search/?ignore_preferences=1&filter=globaltopsellers&page=%d&supportedlang=tchinese&os=win&ndl=1'%pagenum,headers=HEADERS)
        soup = BeautifulSoup(r.text, 'lxml')
        soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/"),class_="search_result_row ds_collapse_flag")
        for i in soups:
            link = i['href']
            ID_app = re.findall(r'/app/(\d+)/', link)
            ID_sup = re.findall(r'/sub/(\d+)/', link)
            if ID_app:
                ID = ID_app[0]
            elif ID_sup:
                ID = ID_sup[0]
            else:
                ID = None
            linklist.append(link)
            IDlist.append(ID)
        print('已完成'+str(pagenum)+'頁,目前共'+str(len(linklist)))
    return linklist,IDlist

def gameprice(n): 
    finalprices=[]
    originalprices=[]
    for pagenum in range(1,n+1):
        r = requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&os=win&filter=globaltopsellers&page=%d'%pagenum,headers=HEADERS)
        soup = BeautifulSoup(r.text, 'lxml')
        name = soup.find_all(class_='responsive_search_name_combined')
        for i in name:
            a_tags1 = i.find_all(class_="discount_final_price")
            a_tags2 = i.find_all(class_="discount_original_price")
            if not a_tags1:
                k='免費遊玩'
                finalprices.append(k)
            for n in a_tags1:
                price1 = n.get_text(strip=True)
                k = str(price1)
                finalprices.append(k)
            if not a_tags2:
                k2=None
                originalprices.append(k2)
            for m in a_tags2:
                price2 = m.get_text(strip=True)
                k2 = str(price2)
                originalprices.append(k2)
    return originalprices, finalprices

def getdf(n): #轉df
    linklist,IDlist = getgamelist(n)
    originalprices, finalprices = gameprice(n)
    df = pd.DataFrame(list(zip(linklist,IDlist,originalprices,finalprices)),columns =['Link', 'ID', '原始價格', '最終價格'])
    return df
if __name__ == "__main__":
    df = getdf(n) 
    df['ID'] = df['ID'].astype('str') 
    df.index = df.index + 1
    df.to_excel(path) 

已完成1頁,目前共25
已完成2頁,目前共50
已完成3頁,目前共75
已完成4頁,目前共100
已完成5頁,目前共125
已完成6頁,目前共150
已完成7頁,目前共175
已完成8頁,目前共200
已完成9頁,目前共225
已完成10頁,目前共250
已完成11頁,目前共275
已完成12頁,目前共300
已完成13頁,目前共325
已完成14頁,目前共350
已完成15頁,目前共375
已完成16頁,目前共400
已完成17頁,目前共425
已完成18頁,目前共450
已完成19頁,目前共475
已完成20頁,目前共500
已完成21頁,目前共525
已完成22頁,目前共550
已完成23頁,目前共575
已完成24頁,目前共600
已完成25頁,目前共625
已完成26頁,目前共650
已完成27頁,目前共675
已完成28頁,目前共700
已完成29頁,目前共725
已完成30頁,目前共750
已完成31頁,目前共775
已完成32頁,目前共800
已完成33頁,目前共825
已完成34頁,目前共850
已完成35頁,目前共875
已完成36頁,目前共900
已完成37頁,目前共925
已完成38頁,目前共950
已完成39頁,目前共975
已完成40頁,目前共1000
已完成41頁,目前共1025
已完成42頁,目前共1050
已完成43頁,目前共1075
已完成44頁,目前共1100
已完成45頁,目前共1125
已完成46頁,目前共1150
已完成47頁,目前共1175
已完成48頁,目前共1200
已完成49頁,目前共1225
已完成50頁,目前共1250
已完成51頁,目前共1275
已完成52頁,目前共1300
已完成53頁,目前共1325
已完成54頁,目前共1350
已完成55頁,目前共1375
已完成56頁,目前共1400
已完成57頁,目前共1425
已完成58頁,目前共1450
已完成59頁,目前共1475
已完成60頁,目前共1500
已完成61頁,目前共1525
已完成62頁,目前共1550
已完成63頁,目前共1575
已完成64頁,目前共1600
已完成65頁,目前共1625
已完成66頁,目前共1650
已完成67頁,目前共1675
已完成68頁,目前共1700
已完成69頁,目前共1725
已完成70頁,目前共1750
已

In [7]:
def gamename(soup):  #遊戲名字
    try:
        a = soup.find(class_="apphub_AppName")
        k = str(a.string)
    except:
        a = soup.find(class_="pageheader")
        k = str(a.string)
    return k

def taglist(soup):  #標籤列表
    list1=[]
    a = soup.find_all(class_="app_tag")
    for i in a:
        k = str(i.string).replace('	', '').replace('\n', '').replace('\r', '')
        if k == '+':
            pass
        else:
            list1.append(k)
    list1 = str('\n'.join(list1))
    return list1

def description(soup):  #遊戲描述
    a = soup.find(class_="game_description_snippet")
    k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '')
    return k

def reviewsummary(soup):   #總體評價
    a = soup.find(class_="summary column")
    try:
        k = str(a.span.string)
    except:
        k=str(a.text)
    return k

def getdate(soup):   #發行日期
    a = soup.find(class_="date")
    k = str(a.string)
    return k

def userreviewsrate(soup):   #總體數量好評率
    a = soup.find(class_="user_reviews_summary_row")
    k = str((a.attrs)['data-tooltip-html'])
    return k

def developer(soup):   #開發商
    a = soup.find(id="developers_list")
    k = str(a.a.string)
    return k

def getdetail(x):
    tag, des, reviews, date, rate, dev, review, name, =' ', ' ', ' ', ' ', ' ', ' ',' ',' '
    global count
    try:
        r = requests.get(x['Link'], headers=HEADERS,timeout=10)
    except:
        print('伺服器無回應1')
        try:
            r = requests.get(x['Link'], headers=HEADERS,timeout=10)
        except:
            print('伺服器無回應2')
            try:
                r = requests.get(x['Link'], headers=HEADERS,timeout=10)
            except:
                print('伺服器無回應3')
    try:
        soup = BeautifulSoup(r.text, 'lxml')
        name = gamename(soup)
        tag = taglist(soup)
        des = description(soup)
        reviews = reviewsummary(soup)
        date = getdate(soup)
        rate = userreviewsrate(soup)
        dev = developer(soup)
        print('已完成: '+name+' '+ str(x['ID']) +'第%d個'%count)
    except:
        print('未完成: '+ str(x['ID']) +'第%d個'%count)
    count += 1
    return name,tag,des,reviews,date,rate,dev

if __name__ == "__main__":
    df1 = pd.read_excel('steam_Link&ID.xlsx')
    count = 1
    df1['詳細'] = df1.apply(lambda x: getdetail(x), axis=1)
    df1['名字'] = df1.apply(lambda x: x['詳細'][0], axis=1)
    df1['標籤'] = df1.apply(lambda x: x['詳細'][1], axis=1)
    df1['描述'] = df1.apply(lambda x: x['詳細'][2], axis=1)
    df1['近期評價'] = df1.apply(lambda x: x['詳細'][3], axis=1)
    df1['發行日期'] = df1.apply(lambda x: x['詳細'][4], axis=1)
    df1['近期數量好評率'] = df1.apply(lambda x: x['詳細'][5], axis=1)
    df1['開發商'] = df1.apply(lambda x: x['詳細'][6], axis=1)
    df1.to_excel('games_data1.xlsx',index = False)
    print('已完成全部')

已完成: 機戰傭兵™VI 境界天火™ 1888160第1個
已完成: Counter-Strike: Global Offensive 730第2個
已完成: Baldur's Gate 3 1086940第3個
已完成: 天命2 1085660第4個
已完成: 《Apex 英雄》 1172470第5個
已完成: PUBG: BATTLEGROUNDS 578080第6個
已完成: 《決勝時刻®》 1938090第7個
未完成: 1675200第8個
已完成: NARAKA: BLADEPOINT 1203220第9個
已完成: Forza Horizon 5 1551360第10個
已完成: Saints Row 742420第11個
已完成: Wayfinder 1171690第12個
已完成: 翼星求生 ICARUS 1149460第13個
已完成: Cyberpunk 2077 1091500第14個
已完成: Dota 2 570第15個
已完成: War Thunder 236390第16個
已完成: Grand Theft Auto V 271590第17個
已完成: Rust 252490第18個
已完成: Team Fortress 2 440第19個
未完成: 2138330第20個
已完成: Warframe 230410第21個
已完成: Tom Clancy's Rainbow Six® Siege 359550第22個
已完成: Blasphemous 2 2114740第23個
已完成: 緋染天空 Heaven Burns Red 1973710第24個
已完成: 雀魂麻將(MahjongSoul) 1329410第25個
已完成:  《Madden NFL 24》 2140330第26個
未完成: 1648532第27個
已完成: 進擊羔羊傳說 1313140第28個
已完成: GUILTY GEAR -STRIVE- 1384160第29個
未完成: 2378500第30個
已完成: Dead by Daylight 381210第31個
已完成: 艾爾登法環 1245620第32個
已完成: Black Desert 582660第33個
已完成: No Man's Sky 275850第34個
已完成: Party Animal

已完成: RPG Maker MV 363890第257個
已完成: GTFO 493520第258個
未完成: 1314563第259個
未完成: 1118010第260個
已完成: コイカツ！ / Koikatsu Party 1073440第261個
已完成: ONE PIECE海賊無雙4 1089090第262個
已完成: Portal 2 620第263個
未完成: 2536380第264個
未完成: 1746860第265個
已完成: 雲霄飛車之星 493340第266個
已完成: Stronghold Kingdoms 47410第267個
已完成: Floodland 1336180第268個
已完成: 蓋婭迷航 2273170第269個
已完成: 地球不屈 Terra Invicta 1176470第270個
已完成: Tormented Souls 1367590第271個
已完成: Aliens: Dark Descent 1150440第272個
已完成: eFootball™ 2023 1665460第273個
已完成: DJMAX RESPECT V 960170第274個
已完成: Action對魔忍 1335200第275個
已完成: 歧路旅人 II 1971650第276個
已完成: Total War: WARHAMMER 364360第277個
已完成: Universe Sandbox 230290第278個
已完成: S.T.A.L.K.E.R. 2: Heart of Chornobyl 1643320第279個
已完成: Super Bunny Man 673750第280個
已完成: ISEKAI FRONTLINE 1924140第281個
已完成: Aimlabs 714010第282個
已完成: Muse Dash 774171第283個
已完成: 原子之心 668580第284個
已完成: Trine 5: A Clockwork Conspiracy 1436700第285個
已完成: Borderlands 2 49520第286個
已完成: Slay the Spire 646570第287個
未完成: 2015890第288個
已完成: Minecraft Dungeons 1672970第289個
已

已完成: S.T.A.L.K.E.R.: Shadow of Chernobyl 4500第525個
已完成: Return of the Obra Dinn 653530第526個
已完成: S.T.A.L.K.E.R.: Shadow of Chernobyl 4500第527個
已完成: Tribes of Midgard 858820第528個
已完成: Holdfast: Nations At War 589290第529個
已完成: 旅者之憩 Travellers Rest 1139980第530個
已完成: FIVE NIGHTS AT FREDDY'S: HELP WANTED 732690第531個
已完成: War Planet Online: Global Conquest 719640第532個
已完成: Persona 4 Golden 1113000第533個
已完成: 符文工廠3豪華版 2243710第534個
已完成: Red Dead Online 1404210第535個
未完成: 2290790第536個
已完成: Shadow Tactics: 愛子的抉擇 Aiko's Choice 1579380第537個
已完成: Teenage Mutant Ninja Turtles: Shredder's Revenge 1361510第538個
未完成: 225340第539個
已完成: 牆世界 2187290第540個
已完成: A Total War Saga: TROY 1099410第541個
已完成: Middle-earth™: Shadow of War™ 356190第542個
未完成: 830450第543個
已完成: We Were Here Together 865360第544個
已完成: CONFLICT OF NATIONS: WORLD WAR 3 784950第545個
已完成: Two Point Hospital 535930第546個
已完成: Flashing Lights - 警情，消防，急救 605740第547個
已完成: Road 96: Mile 0 🛹 2095340第548個
已完成: Bus Simulator 21 Next Stop 976590第549個
已完成: As

已完成: Monster Hunter Stories 2: 破滅之翼 1277400第790個
已完成: CRISIS CORE –FINAL FANTASY VII– REUNION 1608070第791個
已完成: 三國群英傳8 875210第792個
未完成: 2475260第793個
已完成: 女神異聞錄５ 亂戰：魅影攻手 1382330第794個
已完成: 漫威午夜之子 368260第795個
已完成: Spelunky 2 418530第796個
未完成: 1829590第797個
已完成: Katana ZERO 460950第798個
已完成: Wargame: Red Dragon 251060第799個
已完成: 規則支配 2528120第800個
已完成: 傳送地下城 1679220第801個
已完成: OVR Toolkit 1068820第802個
未完成: 2052720第803個
未完成: 1389240第804個
已完成: 尋找家園：末日基地 2093000第805個
未完成: 1324970第806個
已完成: NBA 2K22 1644960第807個
未完成: 2363740第808個
已完成: VR Kanojo / VRカノジョ 751440第809個
未完成: 1308200第810個
已完成: Doki Doki Literature Club Plus! 1388880第811個
已完成: Age of Empires: Definitive Edition 1017900第812個
未完成: 1458151第813個
未完成: 1934140第814個
已完成: Incredibox 1545450第815個
已完成: Fabledom 1651560第816個
已完成: Ghostwire: Tokyo 1475810第817個
已完成: Mindustry 1127400第818個
已完成: 蜜蜂島 2345020第819個
已完成: LOST EMBER - VR Edition 1880890第820個
未完成: 2504240第821個
已完成: Lost Ruins 1306630第822個
已完成: Farmer's Dynasty 678900第823個
已完成: 光明記憶：無限 1178830第

未完成: 1668110第1067個
未完成: 591800第1068個
未完成: 2416900第1069個
未完成: 2222570第1070個
已完成: Goat Simulator 265930第1071個
已完成: Sword Art Online: Fatal Bullet 626690第1072個
未完成: 1977570第1073個
已完成: 100% Orange Juice 282800第1074個
已完成: LIVE A LIVE 2014380第1075個
未完成: 2154172第1076個
已完成: 《極速快感™：超熱力追緝》重製版 1328660第1077個
已完成: Lumberjack's Dynasty 968970第1078個
已完成: Forsaken World: Gods and Demons 1708330第1079個
已完成: Kingdom Rush  - Tower Defense 246420第1080個
未完成: 1724940第1081個
已完成: Marble It Up! Ultra 864060第1082個
未完成: 2353720第1083個
已完成: Assassin’s Creed® Brotherhood 48190第1084個
已完成: Clicker Heroes 363970第1085個
未完成: 1652891第1086個
已完成: Assassin's Creed® Revelations 201870第1087個
已完成: Craft The World 248390第1088個
已完成: BlazBlue Centralfiction 586140第1089個
未完成: 1247710第1090個
未完成: 1734080第1091個
未完成: 2054471第1092個
已完成: HARVESTELLA 1816300第1093個
未完成: 1493250第1094個
已完成: Might & Magic® Heroes® VII 321960第1095個
已完成: Pikuniku 572890第1096個
已完成: HARD BULLET 1294760第1097個
已完成: 軒轅劍參 雲和山的彼端 1638230第1098個
已完成: 魅魔咖啡廳 1520500第1099個

...

已完成: 信長之野望･大志 628890第2474個
已完成: Mega Man X Legacy Collection 2 743900第2475個
未完成: 2076055第2476個
已完成: UFO：不明墜落物 2087610第2477個
未完成: 1561770第2478個
已完成: 神舞幻想 Faith of Danschant 724970第2479個
已完成: King of Crabs 1273710第2480個
已完成: Whateverland 1175840第2481個
未完成: 2104490第2482個
未完成: 1235767第2483個
已完成: 卡德里亞道具屋 883860第2484個
已完成: Mosaique Neko Waifus 5 2165610第2485個
已完成: 紙嫁衣 1735950第2486個
已完成: 高傲的蕾娜絕不屈服 2395680第2487個
已完成: MotoGP™21 1447000第2488個
未完成: 1206340第2489個
未完成: 2150000第2490個
未完成: 992171第2491個
已完成: Cat's Kiss 貓研社 1409770第2492個
已完成: Poozle Mania 2191080第2493個
已完成: AI：夢境檔案 948740第2494個
未完成: 2321094第2495個
已完成: 合戰忍者村物語 (Ninja Village) 1918520第2496個
未完成: 2062376第2497個
未完成: 1248433第2498個
已完成: Life is Feudal: Your Own 290080第2499個
已完成: The Good Life 1452500第2500個
未完成: 1235760第2501個
已完成: 創意漢堡物語 (Burger Bistro Story) 1933990第2502個
已完成: Marvel vs. Capcom: Infinite 493840第2503個
未完成: 739281第2504個
已完成: 蛇香之夜 ～Allure of MUSK～　第一夜　歐洲篇 2129280第2505個
未完成: 2370700第2506個
已完成: House Flipper Pets VR 1839190第2507個

未完成: 1690140第2763個
未完成: 1323320第2764個
已完成: Nightmare of Decay 1848450第2765個
已完成: 軒轅劍伍 一劍凌雲山海情 1728520第2766個
已完成: 閃亂忍忍忍者大戰戰機少女 -少女們的響艷- 1836860第2767個
未完成: 779935第2768個
未完成: 889890第2769個
已完成: HALF DEAD 2 914260第2770個
已完成: Chorus 1153640第2771個
未完成: 2316610第2772個
已完成: 怡紅院 1802980第2773個
未完成: 2156250第2774個
已完成: Oceanhorn: Monster of Uncharted Seas 339200第2775個
未完成: 1248431第2776個
未完成: 1646370第2777個
已完成: 高考戀愛一百天 347620第2778個
已完成: 三國群英傳M 2321430第2779個
已完成: SRPG Studio 857320第2780個
已完成: 軒轅劍外傳 雲之遙 1870150第2781個
已完成: 哆啦A夢 牧場物語 自然王國與和樂家人 1492730第2782個
已完成: Tea For God 1764400第2783個
未完成: 1643354第2784個
已完成: Rush Rally 3 2020860第2785個
未完成: 1918190第2786個
未完成: 1585812第2787個
已完成: 停車行不行 837880第2788個
未完成: 530610第2789個
未完成: 2231950第2790個
未完成: 1711182第2791個
未完成: 1093730第2792個
已完成: PAC-MAN MUSEUM+ 1665130第2793個
已完成: 新次元遊戲 戰機少女ＶⅡＲ 774511第2794個
已完成: 三國群英傳2 1437830第2795個
已完成: 透明人間人妻中出 1447310第2796個
已完成: Groupy 2 2296360第2797個
已完成: 滿是色情大姐姐宿舍管理員 2222370第2798個
已完成: Virtual Villagers Origins 2 949050第2799個
已完成: Obse

已完成: Peter的英語冒險 [英語學習] 1726360第3063個
已完成: SD 新 KAMEN RIDER 亂舞 1953070第3064個
未完成: 693170第3065個
已完成: Doll Explorer 2164930第3066個
已完成: 命途 1483370第3067個
已完成: RutonyChat 524660第3068個
未完成: 934690第3069個
未完成: 1279740第3070個
未完成: 1229320第3071個
已完成: Pupperazzi 1028350第3072個
已完成: Zodiacats 1817000第3073個
已完成: Boba Simulator : Idle Shop Management 1847510第3074個
已完成: Fun Hospital 829420第3075個
未完成: 813390第3076個
已完成: Banners of Ruin 1075740第3077個
已完成: 冥婚交友中心《厲鬼的戀愛指南》 1683430第3078個
未完成: 1599770第3079個
未完成: 779934第3080個
未完成: 1011890第3081個
已完成: Pawnbarian 1142080第3082個
已完成: Mini Healer 955740第3083個
已完成: Juicy Realm 732370第3084個
未完成: 1831450第3085個
未完成: 1852255第3086個
未完成: 1241271第3087個
未完成: 425840第3088個
已完成: Ero Manager 1901340第3089個
已完成: OPUS：龍脈常歌 -最終版- 1504500第3090個
未完成: 1108930第3091個
已完成: Bless Global 2094220第3092個
未完成: 1812150第3093個
已完成: The Knight Witch 1872680第3094個
已完成: 多愁善感死亡輪迴 2346580第3095個
已完成: 失落史詩 LOST EPIC 1426490第3096個
未完成: 1944341第3097個
已完成: 賽馬牧場物語 (Pocket Stables) 1918530第3098個
已完成: 賞金列車 - Bo

未完成: 2139265第3358個
已完成: The Last Campfire 990630第3359個
未完成: 1458153第3360個
已完成: Pixel Studio - pixel art editor 1204050第3361個
已完成: STRIKERS 1945 1261960第3362個
已完成: 貓次郎 2157340第3363個
未完成: 2307480第3364個
已完成: 《百英雄傳: 崛起》Eiyuden Chronicle: Rising 1658290第3365個
已完成: Creaks 956030第3366個
未完成: 1996310第3367個
已完成: Train Fever 304730第3368個
未完成: 1051620第3369個
已完成: Hunting Simulator 585080第3370個
已完成: Gravewood High 1239300第3371個
未完成: 2133149第3372個
未完成: 1424060第3373個
未完成: 239391第3374個
已完成: 籃球熱潮物語 (Basketball Club Story) 2102490第3375個
未完成: 1386611第3376個
未完成: 1475880第3377個
已完成: LOVE³ -愛立方- 939600第3378個
未完成: 1087424第3379個
已完成: 阿優拉！危！ 2092260第3380個
已完成: Redout: Enhanced Edition 517710第3381個
未完成: 1664540第3382個
未完成: 1098300第3383個
已完成: [Chilla's Art] Stigmatized Property | 事故物件 1162680第3384個
未完成: 457840第3385個
未完成: 844190第3386個
未完成: 315151第3387個
已完成: Voyage 1051950第3388個
已完成: Idle Taoist Mage Warrior 2427360第3389個
未完成: 2054120第3390個
未完成: 363990第3391個
未完成: 611421第3392個
已完成: YUME : Special Edition 1864070第3393個

未完成: 315150第3665個
未完成: 2441410第3666個
未完成: 1341590第3667個
未完成: 1262571第3668個
已完成: 觅仙 2285570第3669個
未完成: 1767690第3670個
未完成: 2125560第3671個
未完成: 1317160第3672個
已完成: Shovel Knight: Specter of Torment 589510第3673個
未完成: 2167450第3674個
未完成: 1090183第3675個
已完成全部


In [ ]:
#全部爬蟲
import time, re, os
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

def getreviews(ID):
    list1 = []
    browser = webdriver.Chrome()
    browser.get("https://steamcommunity.com/app/%s/reviews/?browsefilter=toprated&snr=1_5_100010_&filterLanguage=tchinese"%str(ID))
    try:
        button = browser.find_element(By.CLASS_NAME, "btn_blue_steamui.btn_medium")
        button.click()
    except:
        pass

    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1.5)
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    contents = browser.find_elements(By.CLASS_NAME, 'apphub_CardTextContent')
    for content in contents:
        string=content.text
        pattern = r"(張貼於：\d+,月,\d+,日)(.*)"
        pattern2 = r'搶先體驗版評論\n'
        pattern3 = r'免費獲得的產品\n'
        replacement = r"\1\n\2"
        string2 = re.sub(pattern, replacement, string)
        string3 = re.sub(pattern2,'', string2)
        adjust_text = re.sub(pattern3,'', string3)
        list1.append(adjust_text)
    browser.quit()
    return list1

path = "C:\\Users\\fento\\Downloads\\data_commendinfo\\games_data1.xlsx"
df_id = pd.read_excel(path)

for index, row in df_id.iterrows():
    game_ID = row['ID']
    list1 = getreviews(str(game_ID))
    review_df = pd.DataFrame(list(zip(list1)),columns =['評論'])
    print(game_ID)
    
    filename = f"C:\\Users\\fento\\Downloads\\data_cleaned\\games'data2.xlsx"
    review_df.to_excel(filename, index=False)
    print(f"File saved: {excel_file_path}")

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time, re

def remove_illegal_chars(val):
    # 此正則表達式會匹配任何非打印字符，您也可以自定義以符合您的需求
    illegal_re = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
    return illegal_re.sub('', str(val))

def getreviews(ID):
    list1 = []
    browser = webdriver.Chrome()
    browser.get("https://steamcommunity.com/app/%s/positivereviews/?browsefilter=toprated&snr=1_5_100010_&filterLanguage=tchinese"%str(ID))
    try:
        button = browser.find_element(By.CLASS_NAME, "btn_blue_steamui.btn_medium")
        button.click()
    except:
        pass

    last_height = browser.execute_script("return document.body.scrollHeight")
    for i in range(5):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    contents = browser.find_elements(By.CLASS_NAME, 'apphub_CardTextContent')
    for content in contents:
        string=content.text
        pattern = r"(張貼於：.+日)(.*)"
        pattern2 = r'搶先體驗版評論\n'
        pattern3 = r'免費獲得的產品\n'
        replacement = r"\1\n\2"
        string2 = re.sub(pattern, replacement, string)
        string3 = re.sub(pattern2,'', string2)
        adjust_text = re.sub(pattern3,'', string3)
        list1.append(adjust_text)
    browser.quit()
    return ' '.join(list1)

path = "C:\\Users\\fento\\Downloads\\data_cleaned\\gamesdata_280.xlsx"
df_id = pd.read_excel(path)

all_reviews = []
for index, row in df_id.iterrows():
    game_ID = row['ID']
    print(f"{index} Fetching reviews for game ID: {game_ID}")
    try:
        reviews = getreviews(game_ID)
        reviews = remove_illegal_chars(reviews)  # Remove illegal characters
        all_reviews.append(reviews)
    except Exception as e:
        print(f"An error occurred: {e}")
        all_reviews.append("Error")

df_id['全部評論'] = all_reviews
df_id.to_excel(path, index=False)

0 Fetching reviews for game ID: 1086940
1 Fetching reviews for game ID: 730
2 Fetching reviews for game ID: 1085660
3 Fetching reviews for game ID: 230410
4 Fetching reviews for game ID: 1172470
5 Fetching reviews for game ID: 1938090
6 Fetching reviews for game ID: 578080
7 Fetching reviews for game ID: 1551360
8 Fetching reviews for game ID: 1203220
9 Fetching reviews for game ID: 1149460
10 Fetching reviews for game ID: 359550
11 Fetching reviews for game ID: 236390
12 Fetching reviews for game ID: 271590
13 Fetching reviews for game ID: 742420
14 Fetching reviews for game ID: 440
15 Fetching reviews for game ID: 252490
16 Fetching reviews for game ID: 1313140
17 Fetching reviews for game ID: 1329410
18 Fetching reviews for game ID: 381210
19 Fetching reviews for game ID: 275850
20 Fetching reviews for game ID: 739630
21 Fetching reviews for game ID: 582660
22 Fetching reviews for game ID: 1245620
23 Fetching reviews for game ID: 412020
24 Fetching reviews for game ID: 671860
25 Fet

202 Fetching reviews for game ID: 939400
203 Fetching reviews for game ID: 683320
204 Fetching reviews for game ID: 1151050
205 Fetching reviews for game ID: 931500
206 Fetching reviews for game ID: 958400
207 Fetching reviews for game ID: 1369630
208 Fetching reviews for game ID: 520950
209 Fetching reviews for game ID: 1379130
210 Fetching reviews for game ID: 1162130
211 Fetching reviews for game ID: 552280
212 Fetching reviews for game ID: 381020
213 Fetching reviews for game ID: 1745310
214 Fetching reviews for game ID: 1563130
215 Fetching reviews for game ID: 1761380
216 Fetching reviews for game ID: 2313250
217 Fetching reviews for game ID: 637670
218 Fetching reviews for game ID: 1282210
219 Fetching reviews for game ID: 986680
220 Fetching reviews for game ID: 446560
221 Fetching reviews for game ID: 1178790
222 Fetching reviews for game ID: 677340
223 Fetching reviews for game ID: 1213750
224 Fetching reviews for game ID: 1528560
225 Fetching reviews for game ID: 1222800
226

In [18]:
path = 'C:\\Users\\fento\\Downloads\\games\'data1.xlsx'
df = pd.read_excel(path,index_col=0)
df.info()  #看一下數據結構

def datetransform(x):
    try:
        date_obj = datetime.strptime(x['發行日期'], '%Y 年 %m 月 %d 日')
        datetime64_obj = np.datetime64(date_obj)
    except:
        datetime64_obj = None
    return datetime64_obj

df['發行日期'] = df.apply(lambda x: datetransform(x),axis=1)
df.info()

def getreviewsnum(x):
    x1 = x['近期數量好評率']
    if re.search('過去 30 天發表的 (.*) 篇使用者評論',x1):
        num = re.search('過去 30 天發表的 (.*) 篇使用者評論',x1).group(1)
    elif re.search('(\d*) 篇使用者評論中有 (\d*%) 給予此遊戲好評',x1):
        num = re.search('(\d*) 篇使用者評論中有 (\d*%) 給予此遊戲好評',x1).group(1)
    else:
        num = '0'
    return num
def getreviewsrate(x):
    x = x['近期數量好評率']
    if re.search('使用者評論中有 (\d*%) 給予好評。',x):
        rate = re.search('使用者評論中有 (\d*%) 給予好評。',x).group(1)
    elif re.search('使用者評論中有 (\d*%) 給予此遊戲好評。',x):
        rate = re.search('使用者評論中有 (\d*%) 給予此遊戲好評。',x).group(1)
    else :
        rate=''
    return rate
df['評價數量']=df.apply(lambda x:getreviewsnum(x),axis=1)
df['好評率']=df.apply(lambda x:getreviewsrate(x),axis=1)
df['評價數量'] = df['評價數量'].apply(lambda x:x.replace(',',''))
df['好評率'] = df['好評率'].apply(lambda x:str(x).replace('%',''))
df['評價數量'] = pd.to_numeric(df['評價數量'])
df['好評率'] = pd.to_numeric(df['好評率'])

df['價格'] = np.where(df['原始價格'].notnull(), df['原始價格'], df['最終價格'])
df['價格'] = df['價格'].str.replace(r'NT\$|,|免費遊玩', '', regex=True)
df['價格'] = pd.to_numeric(df['價格']).fillna(0).astype(int)
path2 = 'C:\\Users\\fento\\Downloads\\games\'data2.xlsx'
df.to_excel(path2)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 1 to 2150
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Link     1988 non-null   object
 1   ID       1988 non-null   int64 
 2   原始價格     246 non-null    object
 3   最終價格     1988 non-null   object
 4   詳細       1988 non-null   object
 5   名字       1988 non-null   object
 6   標籤       1988 non-null   object
 7   描述       1988 non-null   object
 8   近期評價     1988 non-null   object
 9   發行日期     1988 non-null   object
 10  近期數量好評率  1988 non-null   object
 11  開發商      1988 non-null   object
 12  評論       1988 non-null   object
dtypes: int64(1), object(12)
memory usage: 217.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 1 to 2150
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Link     1988 non-null   object        
 1   ID       1988 non-null   int64        

,Link,ID,原始價格,最終價格,詳細,名字,標籤,描述,近期評價,發行日期,近期數量好評率,開發商,評論,評價數量,好評率,價格
Unnamed: 0,,,,,,,,,,,,,,,,
1,https://store.steampowered.com/app/1086940/Bal...,1086940,NaN,"NT$ 1,599.00","(""Baldur's Gate 3"", '角色扮演\n選擇導向\n自訂角色\n劇情豐富\n冒...",Baldur's Gate 3,角色扮演\n選擇導向\n自訂角色\n劇情豐富\n冒險\n電腦角色扮演\n線上合作\n回合制戰...,"Baldur’s Gate 3 is a story-rich, party-based R...",壓倒性好評,2023-08-03,"過去 30 天發表的 168,608 篇使用者評論中有 96% 給予好評。",Larian Studios,"奇怪?怎麼早上9點了，剛剛不是早上10點嗎?\n,替,Baldur's,Gate,3,撰寫評...",168608,96.0,1599
2,https://store.steampowered.com/app/730/Counter...,730,NaN,NT$ 462.00,"('Counter-Strike: Global Offensive', '第一人稱射擊\n...",Counter-Strike: Global Offensive,第一人稱射擊\n射擊\n多人\n競技\n動作\n團隊導向\n電競\n戰術\n第一人稱視角\n...,《絕對武力：全球攻勢》（CS: GO）延續了原作自 1999 年發行以來，在團隊動作類遊戲上...,極度好評,2012-08-22,"過去 30 天發表的 75,103 篇使用者評論中有 89% 給予好評。",Valve,很需要点数去换些色色的头像摸一下猫猫会张嘴，一个赞摸一次！！！⣿⣿⣿⠟⠛⠛⠻⣿⣿⣿⣿⣿⣿⣿⣿...,75103,89.0,462
3,https://store.steampowered.com/app/1888160/VI/...,1888160,NaN,"NT$ 1,790.00","('機戰傭兵™VI 境界天火™', '機甲\n3D\n自訂角色\n第三人稱視角\n戰鬥\n動...",機戰傭兵™VI 境界天火™,機甲\n3D\n自訂角色\n第三人稱視角\n戰鬥\n動作\n機器人\n末日之後\n快節奏\n...,《機戰傭兵™VI 境界天火™》是萬代南夢宮娛樂與FromSoftware運用歷經多款動作遊戲...,極度好評,2023-08-25,"15,465 篇使用者評論中有 82% 給予此遊戲好評。",FromSoftware Inc.,神作!!可以享受高速戰鬥的快感，隻身一人幹翻整個艦隊，難度適中，沒有堆怪BOSS，基本上都是...,465,82.0,1790
4,https://store.steampowered.com/app/1085660/2/?...,1085660,NaN,免費遊玩,"('天命2', '免費遊玩\n開放世界\n第一人稱射擊\n刷寶射擊\nMMORPG\n合作\...",天命2,免費遊玩\n開放世界\n第一人稱射擊\n刷寶射擊\nMMORPG\n合作\n玩家對戰\n玩家...,《天命2》是一部動作類大型多人線上遊戲、一個會持續進步的遊戲世界，讓你和朋友們隨時隨地一起免...,褒貶不一,2019-10-01,"過去 30 天發表的 5,876 篇使用者評論中有 57% 給予好評。",Bungie,"沒朋友跟我玩,嗚嗚\n煩餒,每個禮拜更新完,都進不去,都要重新安裝,才能進遊戲,是在搞什麼啦...",5876,57.0,0
5,https://store.steampowered.com/app/230410/Warf...,230410,NaN,免費遊玩,"('Warframe', '免費遊玩\n動作角色扮演\n刷寶射擊\n第三人稱射擊\n動作\n...",Warframe,免費遊玩\n動作角色扮演\n刷寶射擊\n第三人稱射擊\n動作\n角色扮演\n第三人稱視角\n...,在這個情節引人入勝的免費線上動作遊戲中化身成為一名勢不可擋的戰士甦醒過來並與你的好友們並肩作...,極度好評,2013-03-26,"過去 30 天發表的 3,861 篇使用者評論中有 83% 給予好評。<br><br>此產品...",Digital Extremes,"在這裡養了一隻貓貓，路過的朋友可以摸摸她一讚=摸一次 , ,＿＿ , ／＞ ...",3861,83.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146,https://store.steampowered.com/app/1158980/_/?...,1158980,NaN,免費遊玩,"('放置大魔王', '免費遊玩\n策略\n休閒\n放置\n獨立\n冒險\n點擊遊戲\n卡牌遊...",放置大魔王,免費遊玩\n策略\n休閒\n放置\n獨立\n冒險\n點擊遊戲\n卡牌遊戲\n色情內容\n搶先...,Idle Big Devil is a free idle RPG game with 3D...,極度好評,2021-11-03,"3,808 篇使用者評論中有 83% 給予此遊戲好評。","HONG KONG SPARK SOFTWARE TECHNOLOGY CO., LIMITED",最新的更新把我列的缺點都解決了每個禮拜還會送一張代金券===================...,808,83.0,0
2147,https://store.steampowered.com/app/2164930/Dol...,2164930,NaN,NT$ 186.00,"('Doll Explorer', '策略\n回合制戰術\n輕度 Rogue\n解謎\n卡牌...",Doll Explorer,策略\n回合制戰術\n輕度 Rogue\n解謎\n卡牌遊戲\n牌組製作\n可愛\n2D\n休...,Doll Explorer is a turn-based strategy game in...,好評,2023-08-11,39 篇使用者評論中有 94% 給予此遊戲好評。,Pico Games,"當年Doll,Explorer,Prologue剛出我就一直在等，如今終於推出了正式版，只能...",39,94.0,186
2148,https://store.steampowered.com/app/1233800/JET...,1233800,NaN,NT$ 328.00,"('神田川JET GIRLS', '競速\n日本動畫\n色情內容\n自訂角色\n動作\n女主...",神田川JET GIRLS,競速\n日本動畫\n色情內容\n自訂角色\n動作\n女主人翁\n劇情豐富\n裸露\n可愛,動畫×遊戲多方面跨媒體發展的爆乳、高速動作競技！！ 《閃亂神樂》、《VALKYRIE DRI...,極度好評,2020-08-26,275 篇使用者評論中有 85% 給予此遊戲好評。,SHADE Inc.,"以前有看過動畫版,現在來玩玩這個遊戲操作比想像的多,一開始上手需要用一點時間,但勝在劇情難度...",275,85.0,328
